<a href="https://colab.research.google.com/github/andre-n-alcantara/IC/blob/main/submit-form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium
!apt-get update

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 30.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,190 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-

In [ ]:
import requests
import os
import re
import time
import random
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select # Import Select class
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
base_url ='https://olimpiada.ic.unicamp.br/pratique/'
root = './drive/MyDrive/sol'
def get_url(base_url, file):
    pattern = re.compile(r'\d{4}')
    year = pattern.search(file).group()
    pattern = re.compile(r'p.')
    nivel = pattern.search(file).group()
    pattern = re.compile(r'f.-.*$')
    group = pattern.search(file).group()
    fase,name = group.split('-',1)
    name = name[:-4]
    if nivel=='pu':
        nivel='ps'
    return base_url+f'{nivel}/{year}/{fase}/{name}/'




def submit_file(base_url, file_path):
    file_path = os.path.abspath(file_path)
    time.sleep(random.uniform(5, 10))
    # Use selenium to handle delayed redirects
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')
    driver = webdriver.Firefox(options=options)

    try:
        driver.get(base_url)  # Navigate to the initial redirect URL
    except WebDriverException:
        print(f"{base_url} URL not found")
        driver.quit()
        return
    select_elem = None
    try:
        select_elem = Select(driver.find_element(By.TAG_NAME, 'select'))
    except NoSuchElementException:
        if 'pu/' in base_url:
            print(f'{base_url} URL not found')
            return
        base_url=base_url.replace('ps/','pu/')
        submit_file(base_url, file_path)
        return
    for option in select_elem.options:
        if 'g++' in option.text or 'C++' in option.text:
            option.click()
            break
    file_input = driver.find_element(By.NAME, 'data')
    file_input.send_keys(file_path)

    driver.find_element(By.TAG_NAME, 'form').submit()
    driver.delete_all_cookies()
    # Wait for the final redirect (adjust timeout as needed)
    try:
        final_url = WebDriverWait(driver, 60).until(
            EC.url_contains('resultado')  # Wait for URL to change
        )
        center_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'inner-page'))
        )
        file_path = file_path[:-4]
        file_path += '.txt'
        with open(os.path.join(file_path), "w", encoding='utf-8') as f:
            f.write(center_element.text.split('Tarefas Programação',1)[0])
        print(f"{file_path} File saved successfully.")
    except TimeoutException:
        print(f"{file_path} Timeout waiting for redirect.")

    driver.quit()  # Close the browser

cnt = 0
for file in os.listdir(root):
    if not file.endswith('.cpp') or os.path.exists(os.path.join(root, file[:-4]+'.txt')):
        continue
    if cnt > 30:
        break
    cnt += 1
    url = get_url(base_url, file)
    submit_file(url, os.path.join(root, file))

/content/drive/MyDrive/sol/2013-p1-f1-corrida.txt File saved successfully.
/content/drive/MyDrive/sol/2017-pu-f3-carrinho.txt File saved successfully.
/content/drive/MyDrive/sol/2013-pj-f1-tomadas.cpp Timeout waiting for redirect.
/content/drive/MyDrive/sol/2018-p2-f1-compensacao.txt File saved successfully.
/content/drive/MyDrive/sol/2014-pj-f1-gangorra.txt File saved successfully.
/content/drive/MyDrive/sol/2018-p2-f3-muro.txt File saved successfully.
/content/drive/MyDrive/sol/2018-pu-f3-maximin.txt File saved successfully.
/content/drive/MyDrive/sol/2014-p2-f2-tapetes.txt File saved successfully.
/content/drive/MyDrive/sol/2015-p2-f1-banco.txt File saved successfully.
/content/drive/MyDrive/sol/2014-p2-f1-copa.txt File saved successfully.
/content/drive/MyDrive/sol/2016-pu-f2-times.txt File saved successfully.
/content/drive/MyDrive/sol/2013-p2-f1-catalogo.txt File saved successfully.
/content/drive/MyDrive/sol/2014-pu-f1-carteiro.txt File saved successfully.
/content/drive/MyDrive

In [ ]:
root = './drive/MyDrive/sol'
for file in os.listdir(root):
    if file.endswith('.txt'):
      with open(os.path.join(root, file), "r", encoding='utf-8') as f:
        if 'Pontuação' not in f.read():
            print(f'{file} sem resultado.')